# A Pythonic Object

Python data model enbles users to define types that behave as naturally as the built-in types. You can call built-in functions like `repr(), bytes()` on the instances of those types.

Chpater overview
* build-in functions for user-defined types/objects
* implement an alternative constructor as a class method
* read-only access to attributes
* hash an object
* member variables based on slots

In [1]:
# prepare the environment
%load_ext autoreload
%autoreload 2

## Object Representations

* `repr()`: return a string representing the object for the developer.
* `str()`: return a string representing the object for the users.

## Vector Class Redux

[vector2d.py](./vector2d.py)

In [2]:
from vector2d import Vector2dV0

v1 = Vector2dV0(3, 4)
print(v1.x, v1.y)

x, y = v1
print(x, y)
print(v1)

v1_clone = eval(repr(v1))
print('v1 == v1_clone', v1 == v1_clone)

octets = bytes(v1)
print(octets)
print(abs(v1))
print(bool(v1), bool(Vector2dV0(0,0)))

3.0 4.0
3.0 4.0
(3.0, 4.0)
v1 == v1_clone True
b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
5.0
True False


## An Alternative Constructor

Construct a `Vector2d` from a binary sequence.

```
@classmethod
def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)
```

In [3]:
from vector2d import Vector2dV1

v1 = Vector2dV0(3, 4)
octets = bytes(v1)
v2 = Vector2dV1.frombytes(octets)
print(v2)

(3.0, 4.0)


### classmethod Versus staticmethod

* `classmethod`: operates on the class. The first paremeter is the class itself.
* `staticmethod`: receive no special argument.

In [4]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args
    
print(Demo.klassmeth())
print(Demo.klassmeth('spam'))
print(Demo.statmeth())
print(Demo.statmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


## Formatted Displays

`format()` and `str.format()` delegate the actual formatting to each type by calling their `.__format__(format_spec)` method.

See [Format Specification Mini-Language](https://docs.python.org/3/library/string.html#format-specification-mini-language)

In [5]:
brl = 1 / 2.43
print(brl)
print(format(brl, '0.4f'))
print('1 BRL = {rate:0.2f} USD'.format(rate=brl))

0.4115226337448559
0.4115
1 BRL = 0.41 USD


In [6]:
print(format(42, 'b'))
print(format(2/3, '.1%'))

101010
66.7%


In [7]:
from datetime import datetime
now = datetime.now()
print(format(now, '%H:%M:%S'))
print("It's now {:%I:%M %p}".format(now))

12:13:35
It's now 12:13 PM


If a class has no `__format__`, the method inherited from object returns `str(my_object)`. However, passing a format specifier raises `TypeError`.

In [8]:
from vector2d import Vector2dV0
v1 = Vector2dV0(3, 4)
format(v1)

'(3.0, 4.0)'

In [9]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2dV0.__format__

Fix it by implementing `__format__`. See `Vector2dV2` in [vector2d.py](./vector2d.py).

In [10]:
from vector2d import Vector2dV2
v2 = Vector2dV2(3, 4)
print(format(v2, '.2f'))
print(format(v2, '.3e'))
print(format(v2, 'p'))
print(format(v2, '.3ep'))

(3.00, 4.00)
(3.000e+00, 4.000e+00)
<5.0, 0.9272952180016122>
<5.000e+00, 9.273e-01>


## A Hashable Vector2d

In [11]:
from vector2d import Vector2dV3

v1 = Vector2dV3(3, 4)
v2 = Vector2dV3(3.1, 4.2)
print(hash(v1), hash(v2))
print({v1, v2})

7 384307168202284039
{Vector2dV3(3.1, 4.2), Vector2dV3(3.0, 4.0)}


## Saving Space with the __slots__ Class Attribute
By default, Python stores instance attributes in a per-instance `dict` named `__dict__`. `dict` has a singificant memory overhead. Use `__slots__` class attributes saves a lot of memory by changing the dict to a tuple.

In [12]:
from vector2d import Vector2dV3, Vector2dV3Slots
import sys
import resource

NUM_VECTORS = 10**7

def test_mem(cls: type):
    fmt = 'Selected Vector2d type: {.__name__}'
    print(fmt.format(cls))

    mem_init = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    print('Creating {:,} Vector2d instances'.format(NUM_VECTORS))

    vectors = [cls(3.0, 4.0) for i in range(NUM_VECTORS)]

    mem_final = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    print('Initial RAM usage: {:14,}'.format(mem_init))
    print('  Final RAM usage: {:14,}'.format(mem_final))

test_mem(Vector2dV3)
test_mem(Vector2dV3Slots)

Selected Vector2d type: Vector2dV3
Creating 10,000,000 Vector2d instances
Initial RAM usage:     47,882,240
  Final RAM usage:  1,847,603,200
Selected Vector2d type: Vector2dV3Slots
Creating 10,000,000 Vector2d instances
Initial RAM usage:  1,847,603,200
  Final RAM usage:  2,059,440,128
